## Capstone 1 
# San Francisco Bay Water Quality

ref. [Water quality of SF Bay home page](https://sfbay.wr.usgs.gov/access/wqdata/index.html)
     
     

## Part 1 - Data Wrangling

### Tasks

The first step in completing your capstone project is to collect data. Depending on your dataset, you may apply some of the data wrangling techniques that you learned in this unit.

Include answers to these questions in your submission:
   * What kind of cleaning steps did you perform?

   * How did you deal with missing values, if any?

   * Were there outliers, and how did you handle them?


## Data Acquisition

### Water Quality Data

#### Access
   1. Water quality data, 1969 - 2019, requested via query form. No API is available.
   [Expert query](https://sfbay.wr.usgs.gov/access/wqdata/query/expert.html) in three chunks, saved as CSV files
      1. Julian Date < 1999001 <br/>
      2. 1999001 < Julian Date < 2009001 <br/>
      3. Julian Date > 2009001 <br/>

      
**Note**: Water quality data is also available for download from [ScienceBase](https://www.sciencebase.gov/catalog/item/5841f97ee4b04fc80e518d9f); however, that archive includes fewer parameters and is not as up to date as the database at sfbay.wr.usgs.gov.

#### Files
   1. `SFBayWaterQuality1969-1998.csv` 
   2. `SFBayWaterQuality1999-2008.csv`
   3. `SFBayWaterQuality2009-2019.csv`

   
#### Data Format

All files are formatted as CSV (comma-separated values) with 27 columns.

WaterQuality files have two header rows; the second row shows units of measure. 

<small>

```
Date, Time, Station Number, Distance from 36, Depth, Discrete Chlorophyll, Chlorophyll a/a+PHA, Fluorescence, Calculated Chlorophyll, Discrete Oxygen, Oxygen Electrode Output, Oxygen Saturation %, Calculated Oxygen, Discrete SPM, Optical Backscatter, Calculated SPM, Measured Extinction Coefficient, Calculated Extinction Coefficient, Salinity, Temperature, Sigma-t, Nitrite, Nitrate + Nitrite, Ammonium, Phosphate, Silicate
```
```
MM/DD/YYYY, 24 hr., , [km], [meters], [mg/m3], , [volts], [mg/m3], [mg/L], [volts], , [mg/L], [mg/L], [volts], [mg/L], [per meter], [per meter], [psu], [°C], [kg/m3], [µM], [µM], [µM], [µM], [µM]
```
</small>


### Station Location Information

#### Access

Location data for "standard" stations is available from [ScienceBase](https://www.sciencebase.gov/catalog/item/5966abe6e4b0d1f9f05cf551).

However, more complete location data is available in tables at [sfbay.wr.usgs.gov](https://sfbay.wr.usgs.gov/access/wqdata/overview/wherewhen/where.html). These tables include the genral location of each station (by geographical landmark) as well as data for "non-standard" stations which are sampled less often.

These tableswere copied, pasted into a spreadsheet, then exported as CSV. Header fields were edied o remove newlines and several fields were modified to remove artifacts before exporting to CSV format.
#### Files

   1. `SFBayStationLocations.csv`


#### Data Format

The Station Locations file is CSV format with one header row and 5 columns.

<small> 
```
Station Number, General Location, North Latitude, West Longitude, Depth MLW (meters)
```
</small>




## Setup

Import libraries

In [1]:
# Import useful libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import re


## Read in the Water Quality data

In [2]:
# Read in the Water Quality data
# for now, treat eveything as a string
wq_df1 = pd.read_csv('Data/SFBayWaterQuality1969-1998.csv', header=[0,1])
wq_df2 = pd.read_csv('Data/SFBayWaterQuality1999-2008.csv', header=[0,1])
wq_df3 = pd.read_csv('Data/SFBayWaterQuality2009-2019.csv', header=[0,1])

## Combine datasets

The three water quallity DataFrames have identical columns and can easily be concatenated into one file.

In [3]:
# Concatenate water quality DataFrames
wq_df = pd.concat([wq_df1, wq_df2, wq_df3]).reset_index(drop=True)

In [4]:
# Examine the new DataFrame
wq_df.sample(20)

,Date,Time,Station Number,Distance from 36,Depth,Discrete Chlorophyll,Chlorophyll a/a+PHA,Fluorescence,Calculated Chlorophyll,Discrete Oxygen,...,Measured Extinction Coefficient,Calculated Extinction Coefficient,Salinity,Temperature,Sigma-t,Nitrite,Nitrate + Nitrite,Ammonium,Phosphate,Silicate
,MM/DD/YYYY,24 hr.,Unnamed: 2_level_1,[km],[meters],[mg/m3],Unnamed: 6_level_1,[volts],[mg/m3],[mg/L],...,[per meter],[per meter],[psu],[°C],[kg/m3],[µM],[µM],[µM],[µM],[µM]
156689,8/27/2009,1107,12.0,84.45,6.0,NaN,NaN,0.26,4.0,NaN,...,NaN,NaN,26.32,18.94,18.42,NaN,NaN,NaN,NaN,NaN
996,1/27/1970,1057,16.0,63.08,8.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.40,11.80,NaN,NaN,NaN,NaN,NaN,NaN
28525,3/11/1991,1415,9.0,96.79,6.0,NaN,NaN,0.52,3.2,NaN,...,NaN,NaN,17.04,12.86,12.55,NaN,NaN,NaN,NaN,NaN
105807,7/16/2002,1415,8.0,99.77,2.0,NaN,NaN,0.14,3.4,NaN,...,NaN,NaN,11.35,20.89,6.61,NaN,NaN,NaN,NaN,NaN
159044,12/3/2009,1045,15.0,69.99,20.0,NaN,NaN,0.27,3.5,NaN,...,NaN,NaN,30.66,11.76,23.26,NaN,NaN,NaN,NaN,NaN
205746,5/19/2015,738,28.0,23.57,11.0,NaN,NaN,0.66,8.5,NaN,...,NaN,NaN,29.25,16.65,21.19,NaN,NaN,NaN,NaN,NaN
18475,2/28/1989,1118,13.0,78.54,9.0,NaN,NaN,0.73,2.4,NaN,...,NaN,NaN,27.08,10.63,20.68,NaN,NaN,NaN,NaN,NaN
58025,2/28/1995,1407,23.0,39.66,11.0,NaN,NaN,0.64,2.5,NaN,...,NaN,NaN,23.48,13.17,17.45,NaN,NaN,NaN,NaN,NaN
223474,10/27/2017,923,29.0,20.25,1.0,NaN,NaN,NaN,3.6,NaN,...,0.63,NaN,28.85,18.29,20.50,NaN,NaN,NaN,NaN,NaN


We can now ignore the original Water Quality files / DFs and use the concatenated DF containing all data from 1969 to 2019.

### Handle multi-level index for water quality columns

The original Water Quality CSV files had two-row column headers. The second level is units.

```
wq_df.columns
```
<small>

```
MultiIndex([(                             'Date',          'MM/DD/YYYY'),
            (                             'Time',              '24 hr.'),
            (                   'Station Number',  'Unnamed: 2_level_1'),
            (                 'Distance from 36',                '[km]'),
            (                            'Depth',            '[meters]'),
            (             'Discrete Chlorophyll',             '[mg/m3]'),
            (              'Chlorophyll a/a+PHA',  'Unnamed: 6_level_1'),
            (                     'Fluorescence',             '[volts]'),
            (           'Calculated Chlorophyll',             '[mg/m3]'),
            (                  'Discrete Oxygen',              '[mg/L]'),
            (          'Oxygen Electrode Output',             '[volts]'),
            (              'Oxygen Saturation %', 'Unnamed: 11_level_1'),
            (                'Calculated Oxygen',              '[mg/L]'),
            (                     'Discrete SPM',              '[mg/L]'),
            (              'Optical Backscatter',             '[volts]'),
            (                   'Calculated SPM',              '[mg/L]'),
            (  'Measured Extinction Coefficient',         '[per meter]'),
            ('Calculated Extinction Coefficient',         '[per meter]'),
            (                         'Salinity',               '[psu]'),
            (                      'Temperature',                '[°C]'),
            (                          'Sigma-t',             '[kg/m3]'),
            (                          'Nitrite',                '[µM]'),
            (                'Nitrate + Nitrite',                '[µM]'),
            (                         'Ammonium',                '[µM]'),
            (                        'Phosphate',                '[µM]'),
            (                         'Silicate',                '[µM]')],
           )
```
</small>

It will be easier to work with the data if I save the units into a dictionary and change the DataFrame to only have one level of headers.

In [5]:
# create a dictionary of Water Quality parameters and units
wq_units = {}
for param, unit in wq_df.columns:
    if 'Unnamed:' in unit:
        # handle fields with no units
        unit = ''
    wq_units[param] = unit
    
wq_units

{'Date': 'MM/DD/YYYY',
 'Time': '24 hr.',
 'Station Number': '',
 'Distance from 36': '[km]',
 'Depth': '[meters]',
 'Discrete Chlorophyll': '[mg/m3]',
 'Chlorophyll a/a+PHA': '',
 'Fluorescence': '[volts]',
 'Calculated Chlorophyll': '[mg/m3]',
 'Discrete Oxygen': '[mg/L]',
 'Oxygen Electrode Output': '[volts]',
 'Oxygen Saturation %': '',
 'Calculated Oxygen': '[mg/L]',
 'Discrete SPM': '[mg/L]',
 'Optical Backscatter': '[volts]',
 'Calculated SPM': '[mg/L]',
 'Measured Extinction Coefficient': '[per meter]',
 'Calculated Extinction Coefficient': '[per meter]',
 'Salinity': '[psu]',
 'Temperature': '[°C]',
 'Sigma-t': '[kg/m3]',
 'Nitrite': '[µM]',
 'Nitrate + Nitrite': '[µM]',
 'Ammonium': '[µM]',
 'Phosphate': '[µM]',
 'Silicate': '[µM]'}

In [6]:
# Reset the Water Quality column headers
wq_df.columns = wq_units.keys()

wq_df.columns

Index(['Date', 'Time', 'Station Number', 'Distance from 36', 'Depth',
       'Discrete Chlorophyll', 'Chlorophyll a/a+PHA', 'Fluorescence',
       'Calculated Chlorophyll', 'Discrete Oxygen', 'Oxygen Electrode Output',
       'Oxygen Saturation %', 'Calculated Oxygen', 'Discrete SPM',
       'Optical Backscatter', 'Calculated SPM',
       'Measured Extinction Coefficient', 'Calculated Extinction Coefficient',
       'Salinity', 'Temperature', 'Sigma-t', 'Nitrite', 'Nitrate + Nitrite',
       'Ammonium', 'Phosphate', 'Silicate'],
      dtype='object')

### Convert Date/Time columns to DateTime

The initial dataset has a Date column and a Time column, both in non-standard format. It will be useful to have a single DateTime column.

Issues:
   * The initial Date column is type `string`, M/D/YYYY, with no leading zeroes on day or month, possibly with a leading space. Conveniently, `pd.to_datetime` is able to convert this to DateTime format without trouble.
   * The initial Time column is type `int`, with no leading zeroes on the hour. To concatenate this to the Date column, I need it to be type `string`, 0-padded.

When I have two strings, I can concatenate them into a new DateTime column and covert that to DateTime format.

In [7]:
# Convert the Date field to datetime format
# 6/4/2010 => 2019-06-04
wq_df['Date'] = pd.to_datetime(wq_df['Date'])

# Convert back to string
wq_df['Date'] = wq_df['Date'].astype('str')

# convert Time field from int to str
wq_df['Time'] = wq_df['Time'].astype('str')

# 0-pad Time values
wq_df['Time'] = wq_df['Time'].transform(lambda x: x.rjust(4,'0')) 

# create a new DateTime field by concatenating the strings
wq_df['DateTime'] = wq_df['Date'].str.cat(wq_df['Time'],sep=' ')

# convert the new field to DateTime format
wq_df['DateTime'] = pd.to_datetime(wq_df['DateTime'])



In [8]:
# Update the Water Quality units dictionary with the enhanced date data
wq_units['Date'] = 'YYYY-MM-DD'
wq_units['DateTime'] = 'YYYY-MM-DD HH:MM:SS'


In [9]:
# Move the new DateTime column to the front of the DataFrame
cols = list(wq_df.columns)    # get the list of columns
cols = [cols[-1]] + cols[:-1] # rearrange the list

wq_df = wq_df[cols]   # rearrange the columns

wq_df.columns

Index(['DateTime', 'Date', 'Time', 'Station Number', 'Distance from 36',
       'Depth', 'Discrete Chlorophyll', 'Chlorophyll a/a+PHA', 'Fluorescence',
       'Calculated Chlorophyll', 'Discrete Oxygen', 'Oxygen Electrode Output',
       'Oxygen Saturation %', 'Calculated Oxygen', 'Discrete SPM',
       'Optical Backscatter', 'Calculated SPM',
       'Measured Extinction Coefficient', 'Calculated Extinction Coefficient',
       'Salinity', 'Temperature', 'Sigma-t', 'Nitrite', 'Nitrate + Nitrite',
       'Ammonium', 'Phosphate', 'Silicate'],
      dtype='object')

### Remove Columns that are not useful

**Optical Backscatter** 

According to the data dictionary, due to sensor changes and gain differences, this value is only comparable within cruises and may not be comparable between cruises.

Thus, I will remove this column.

In [10]:
wq_df.drop(columns=['Optical Backscatter'], inplace=True)

In [11]:
del wq_units['Optical Backscatter']

Convert Station numbers to strings and remove unnecessary trailing `.0`


In [19]:
wq_df['Station Number2'].astype(str)

0           4.0
1           4.0
2           4.0
3           4.0
4           5.0
          ...  
237056    657.0
237057    657.0
237058    657.0
237059    657.0
237060    657.0
Name: Station Number2, Length: 237061, dtype: object

In [18]:
wq_df['Station Number2'] = [x. else x for x in wq_df['Station Number']]

In [ ]:
# Treat Station numbers as strings unless we're sorting
wq_df['Station Number'] = wq_df['Station Number'].astype('str')


<hr style="border: 5px solid green;">

## Read in the Stations Tables

In [ ]:
# Read in Station locations
st_df = pd.read_csv('Data/SFBayStationLocationsTable.csv', 
                    dtype={'Station Number' : str},
                    header=0)


In [ ]:
st_df.columns

In [ ]:
# Create a list of stations
station_list = st_df['Station Number'].tolist()
print(*station_list) 

In [ ]:
st_df.head()

Many records in the table do not include geographic degrees (they inherit from the station on the row above).

I want to fill in this data.

In [ ]:
# First, extract the degrees from the curent Lat and Long columns
st_df['North Lat Degrees'] = st_df['North Latitude'].str.extract('^(3[78]) ')
st_df['West Long Degrees'] = st_df['West Longitude'].str.extract('^(-12[12]) ')

In [ ]:
# Then fill forward for rows that did not specify degrees
st_df['North Lat Degrees'].fillna(method='ffill', inplace=True)
st_df['West Long Degrees'].fillna(method='ffill', inplace=True)

In [ ]:
# Next, remove the degrees from the columns that contain minutes
st_df['North Lat Minutes'] = st_df['North Latitude'].str.extract('(\d{1,2}\.\d)\'$')
st_df['West Long Minutes'] = st_df['West Longitude'].str.extract('(\d{1,2}\.\d)\'$')

In [ ]:
st_df

Now I can drop the original Latitude and Longitude columns

In [ ]:
st_df.drop(columns=['North Latitude', 'West Longitude', 
                    ], inplace=True)

In [ ]:
# Save the station locations as a file
st_df.to_csv('Data/SFBayStationLocations.csv', index=False)

In [ ]:
station_dict= {}
for row in st_df.itertuples():
    station_dict[row['Station Number']] = ()
    

<hr style="border: 5px solid green;">

## Examine Water Quality Data



In [ ]:
wq_df.sample(10)

### Issue - missing data

Apparently, some of the water quality parameters are not checked for every sample. 

In [ ]:
wq_df.info()

There are no columns that are entirely missing data and no obvious overlap of columns.

Let's investigate further.

In [ ]:
# Look at the portion of the Water Quality DataFrame 
# where a "random" nurient value is not null.
tmp_df = wq_df.loc[wq_df['Phosphate'].notnull()]
tmp_df.sample(20)

It appears that nutrient (e.g. nitrate, phosphate, ammonium, ...) samples are typically only taken near the surface (e.g. 1 - 2 m depth).

(This was confirmed by the USGS contact for the dataset: We can assume nutrient values from surface to bottom are generally the same. "The Bay is well mixed.")

Check if nutrients are sampled at every station. 

In [ ]:
# Extract nutrient data and sort by station

nutrient_df2 = wq_df.loc[:,['Station Number', 'Nitrite',  'Nitrate + Nitrite', 
                           'Ammonium', 'Phosphate', 'Silicate']
                        ]

nutrient_df2['Station Number'] = pd.to_numeric(nutrient_df2['Station Number'])

# convert stations to numeric for sorting
nutrient_df2 = nutrient_df2.sort_values('Station Number').reindex()


In [ ]:
# Plot Nutrients by station

xticks=range(0,37)
ax = nutrient_df2.plot(x='Station Number', linestyle='none', marker='.', logy=True, 
                       figsize=(10,7), xticks=xticks)
# Set the x scale 
ax.set_xlim((0, 40))

# Set the x-axis label
ax.set_xlabel("Station Number")

# Set the y-axis label
ax.set_ylabel("Nutrients, micromoles [µM] per liter")

# Dont cover the data
ax.legend(bbox_to_anchor=(1.15, 1), loc='upper right')

plt.show()

It appears that nutrients are checked at most stations.

**Note**: The apparent "extra" bar in the plot is due to the presence of station number `29.5`.

Confirm nutrient samples by depth.

In [ ]:
# Nutrients by depth

nutrient_df = wq_df.loc[:,['Depth', 'Nitrite',  'Nitrate + Nitrite', 
                           'Ammonium', 'Phosphate', 'Silicate']
                       ].sort_values('Depth'
                       ).reindex()

xticks=(range(0,100,5))
ax = nutrient_df.plot(x='Depth', linestyle='none', marker='.', logy=True, figsize=(9,7), xticks=xticks)
# Set the x scale 
ax.set_xlim((0, 100))

# Set the x-axis label
ax.set_xlabel("Depth, meters")

# Set the y-axis label
ax.set_ylabel("Nutrients, micromoles [µM] per liter")

ax.grid(True)
plt.show()

Given that nutrients are sampled for only a subset of records (primarily shallower depths), I'm going to extract that subset as a separate DataFrame / file.

In [ ]:
# Build a DataFrame containing nutrient sample data
# Only records where at least one nutrient was sampled will be included 

wq_nutrients_df = wq_df[(wq_df.Nitrite > 0) | (wq_df['Nitrate + Nitrite'] > 0) | 
                        (wq_df.Ammonium > 0) | (wq_df.Phosphate > 0) | (wq_df.Silicate > 0) 
                       ].copy()
wq_nutrients_df.info()

#### For this Nutrient DataFrame (file) we don't need to save all columns


In [ ]:
wq_nutrients_df.drop(columns=['Date', 'Time',
                     'Discrete Chlorophyll', 'Chlorophyll a/a+PHA', 
                     'Fluorescence', 'Calculated Chlorophyll', 
                     'Discrete Oxygen', 'Oxygen Electrode Output',
                     'Oxygen Saturation %', 'Calculated Oxygen', 
                     'Discrete SPM', 'Calculated SPM',
                     'Measured Extinction Coefficient', 
                     'Calculated Extinction Coefficient'
                             ], inplace=True)
wq_nutrients_df.head()

## Run some statistics on Water Quality data

In [ ]:
# Are all stations sampled every time?

# convert station numbers to numeric form for sorting
sn = pd.to_numeric(wq_df['Station Number'])
sn_counts = sn.value_counts().sort_index()

sn_counts

In [ ]:
# Station sampling frequency plot  

ax = sn_counts.plot(kind="bar", figsize=(14,8), legend=False, use_index=True)

# Set the x-axis label
ax.set_xlabel("Station")

# Set the y-axis label
ax.set_ylabel("Number of samples")

plt.show()


In [ ]:
# Are all depths sampled every time? How many depths are sampled?
print("Depth statistics\n")

print("Median", wq_df['Depth'].median())
print(wq_df['Depth'].describe())

# Treat depth as a category...
dp = wq_df['Depth'].astype('category')
dp_counts = dp.value_counts().sort_index()

print(dp.describe())
print("\nDepth\t  Times\n\t  sampled")
print(dp_counts)

In [ ]:
# Depth frequency plot  

ax = dp_counts.plot(kind="bar", figsize=(14,8), legend=False, logy=True)
last = ax.xaxis.get_ticklabels()[-1]

# Show every nth tick label on X axis
n = 3  
[t.set_visible(False) for (i,t) in enumerate(ax.xaxis.get_ticklabels()) if i % n != 0]

# but ensure that the last X-axis tick label is visible
last.set_visible(True)

# Set the x-axis label (description)
ax.set_xlabel("Depth")

# Set the y-axis label
ax.set_ylabel("Number of samples")

plt.show()


### Statistics on Numerical parameters
'Discrete Chlorophyll', ... 'Silicate'
 (Columns 6=>)

In [ ]:
# Boxplots for Nuttrient data across all stations
wq_tmp_df = wq_nutrients_df.loc[:,['Nitrite',  'Nitrate + Nitrite', 
                                   'Ammonium', 'Phosphate', 'Silicate']
                                  ].reset_index(drop=True)

ax = wq_tmp_df.plot(kind="box", vert=False, logx=True, figsize=(10,4))

plt.show()

In [ ]:
# Statistics on numerical parameters 
# 'Discrete Chlorophyll', ... 'Silicate'
# (Columns 6=>)

from scipy.stats import zscore

def find_outliers(s):
    """
    Calculate the zscore for a list of values.
    Report on outliers: outside (-threshold < zscore > threshold)
    """
    threshold = 3

    # z-score z=(x-μ)/σ
    zs=zscore(s)
    # How many values meet the criteria for outliers?
    zo = [x for x in zs if (abs(x) > threshold)]
    
    print("")
    if (len(zo) > 0):
        print("-{0} < zscore > {0}".format(threshold))
        print("min\t    {0}".format(round(min(zo),2)))
        print("max\t    {0}".format(round(max(zo),2)))
        print("samples:  {0}".format(len(zo)))

#end_def find_outliers

    
def summary_stats(s):
    """ 
    Calculate summary statistics for a series or list, s 
    returns a dictionary
    """
    
    stats = {
      'count': 0,
      'max': 0,
      'min': 0,
      'mean': 0,
      'median': 0,
      'mode': 0,
      'std': 0,
      'z': (0,0)
    }
    
    stats['count'] = s.count()
    stats['max'] = s.max()
    stats['min'] = s.min()
    stats['mean'] = round(s.mean(),3)
    stats['median'] = s.median()
    stats['mode'] = s.mode()[0]
    stats['std'] = round(s.std(),3)

    
    std3 = 3* stats['std']
    low_z = round(stats['mean'] - (std3),3)
    high_z = round(stats['mean'] + (std3),3)
    stats['z'] = (low_z, high_z)
        
    return(stats)
    
#end_def summary_stats


def plot_one_var(var, d):
    """ 
      Plot one variable against time 
      var: the variable to plot
      d: a DataFrame of var by date
    """
    ax = d.plot(x='DateTime', linestyle='none', marker='.',  
                       figsize=(14,8), legend=False)
    # Set the x-axis label
    ax.set_xlabel("Date")

    # Set the y-axis label
    ax.set_ylabel(var)

    plt.show()
#end_def plot_one_var


def run_stats(s, col):
    """
       Run statistics on a series (column) of data
       s: data series
       col: column name
    """
    stats = {}
    try:
        stats = summary_stats(s)
        print("Summary Statistics\n") 
        for item in stats.items():
            print(item)

    except:
        print("Insufficient data to compute statistics.")
    
    try:
        find_outliers(s)

    except:
        print("Insufficient data to compute statistics.")

    try:
        # plot this column's values by date
        d1 = d.loc[:, ['DateTime', col]]
        plot_one_var(col, d1)
    except:
        print("Not enough data to plot.")

    

    print("____________________________________________\n")
    
    return(stats)
#end_def run_stats

Just for fun, run the statistics across the entire Bay.

In [ ]:

# Ignore the first six columns (dates, times, station number, 
# distance from station 36, and depth)
for idx, col in enumerate(wq_df.columns):
    if (idx <=5):
        continue

    # Extract rows where col is not NaN
    d = wq_df.loc[wq_df[col].notnull()]
    s=d[col] # The data column we want to look at
    print(col)
    if(s.size < 10):
        print("Not enough data to compute statistics: count =", s.size)
        continue

    stats = run_stats(s, col)
#


Most parameters vary substantially across the entire dataset. The presence of many samples with a high zscore implies poor "bucketing" of samples, indicating that I need to calculate  summary statistics by station.

However, it should be noted that the following parameters show little to no variability across the entire dataset:
   * Silicate
   * Temperature
   * Sigma-t
   * Salinity


Calculated clorophyll, SPM, and O2 map well to their discrete values, confirming the calculation algorithm used by USGS. 

My USGS contact has suggested that I ignore the "discrete" values going forward. I will remove these from the dataset.

In [ ]:
wq_df.drop(columns=['Discrete Chlorophyll', 'Discrete Oxygen', 'Discrete SPM'
                   ], inplace=True)

In [ ]:
wq_df.to_csv('Data/SFBayWaterQuality1969-2019.csv', index=False)

### Run the same statistics and plots by station

In [ ]:
station_stats = {}
for station in stations:
    station_str = str(station).strip('.0')
    station_stats[station_str] = {}

    # Extract only records for this station
    st_df = wq_df[wq_df['Station Number'] == station]
    
    for idx, col in enumerate(st_df.columns):
        # Ignore the first six columns (dates, times, station number, 
        # distance from station 36, and depth)
        if (idx <=5):
            continue

        # Exract rows where col is not NaN
        d = st_df.loc[st_df[col].notnull()]
        s=d[col] # The column we want to look at 
        
        print("Station", station_str, col)
        if(s.size < 10):
            print("Not enough data to compute statistics: count =", s.size)
            continue

        stats = run_stats(s, col)

        station_stats[station_str].update({col : stats})

        print(station_stats[station_str])

#end for station in stations



In [ ]:
station_stats
    

In [ ]:
# boxplots for nutrients by station
for station in stations:
    # subset by station 
    tmp_df = wq_nutrients_df[wq_nutrients_df['Station Number'] == station]
    # and then only nutrient columns
    tmp_df = tmp_df.loc[:,['Nitrite',  'Nitrate + Nitrite', 
                                   'Ammonium', 'Phosphate', 'Silicate']
                                  ].reset_index(drop=True)
    print("Station", station)
    try:
        ax = tmp_df.plot(kind="box", vert=False, logx=True, figsize=(10,4))
        plt.show()
    except:
        print("\tNothing to plot\n")
        continue

   

### Issue: Outliers

There are apparent outliers for several parameters at most stations. However, at this point in the analysis, I do not anticipate that they will cause problems. If I need to remove them at a future date, I will be able to locate them.

### Save Water Quality DataFrames to disk

In [ ]:
# Save our work
wq_df.to_csv('Data/SFBayWaterQuality.csv', index=False)

In [ ]:
# Save the nutrient DataFrame as a file
wq_nutrients_df.to_csv('Data/SFBayWaterQualityNutrientData.csv', index=False)

Review first two lines of the new  Water Quality file on disk:
<small>
    
```
DateTime,Date,Time,Station Number,Distance from 36,Destrpth,Chlorophyll a/a+PHA,Fluorescence,Calculated Chlorophyll,Oxygen Electrode Output,Oxygen Saturation %,Calculated Oxygen,Calculated SPM,Measured Extinction Coefficient,Calculated Extinction Coefficient,Salinity,Temperature,Sigma-t,Nitrite,Nitrate + Nitrite,Ammonium,Phosphate,Silicate
1969-04-10 16:15:00,1969-04-10,1615,4.0,119.9,0.5,,,,,,,,,,0.3,13.1,,,,,,
```
</small>

Next time, we can read this data in with
```
wq_df = pd.read_csv('Data/SFBayWaterQuality.csv', 
                    header=0, 
                    parse_dates=['DateTime', 'Date', 'Time'],
                    dtype={'Station Number' : str}
                    )
                    
wq_df = pd.read_csv('Data/SFBayWaterQualityNutrientData.csv', 
                    header=0, 
                    parse_dates=['DateTime', 'Date', 'Time'],
                    dtype={'Station Number' : str}
                    )
```

In [ ]:
# Save the station locations as a file
st_df.to_csv('Data/SFBayStationLocations.csv', index=False)

We will read Station Locations in with
```
st_df = pd.read_csv('Data/SFBayStationLocations.csv', 
                    header=0,
                    dtype={'Station Number' : str}
                    )
```